In [29]:
import os
import pandas as pd
import numpy as np
import statsmodels.api as sm
import pickle

In [30]:
countries = ['Australia','Austria', 'Belgium','Canada','Denmark','Finland','France','Germany',
             'Greece','Hungary','Iceland','Ireland','Italy','Korea','Netherlands','New Zealand',
             'Norway','Portugal','Spain','Sweden','Switzerland','United Kingdom','United States']

In [31]:
T0 = 1965
T1 = 2011

for filename in os.listdir("data"):
    if filename.endswith(".csv"): 
        file_path = os.path.join("data", filename)  
        key = filename.replace(".csv", "") 
        #print(key)
        df = pd.read_csv(file_path, delimiter=';')
        if len(df.columns) < 2:
            df = pd.read_csv(file_path, delimiter=',')
        globals()[key] = df[(df.iloc[:, 0] >= T0) & (df.iloc[:, 0] <= T1)]


In [32]:
datT = total_gov_revenue
datG = total_gov_exp
datY = gdp
daty = gdp_per_capita
datI = interests
datGS = gross_savings
datNS = net_savings
#datBY = debt_gdp_ratio
datBY = dsy_gengov

In [33]:
n = len(countries)
tsy = {}  
gsy = {}
dsy = {}
ismpl = {}
mt = {}

for i in range(n):
    tsy[countries[i]] = datT[countries[i]] / datY[countries[i]]
    tsy[countries[i]].index = datT["Time"]

    gsy[countries[i]] = (datG[countries[i]] - datI[countries[i]]) / datY[countries[i]]
    gsy[countries[i]].index = datT["Time"]

    dsy[countries[i]] = tsy[countries[i]]-gsy[countries[i]]
    dsy[countries[i]].index = datT["Time"]

In [34]:
mt = 0
for i in range(n):
    mt=mt+np.mean(tsy[countries[i]])/n

mt

# x ={}

# for i in range(n):
#     x[countries[i]] = np.sum(tsy[countries[i]])/len(tsy[countries[i]])
#     print(x[countries[i]])

# for i in range(n):
#     alpha = np.mean(x[countries[i]])

# print("alpha = ",alpha)

0.42984464476283923

In [35]:
ALP1 = {}
ALP2 = {}
ALP3 = {}

for i in range(n):
    ALP1[countries[i]] = max(dsy[countries[i]].dropna())
    ALP2[countries[i]] = max(tsy[countries[i]].dropna()) - min(gsy[countries[i]].dropna())
    ALP3[countries[i]] = np.mean(dsy[countries[i]].dropna())

In [36]:
ly = {}
dY = {}

for i in range(n):
    ly[countries[i]] = np.log(daty[countries[i]].dropna().reset_index(drop=True))
    dY[countries[i]] = ly[countries[i]].diff(1)

In [37]:
MDY = {}
RHO = {}
SE = {}
res = {}

for i in range(n):
    model = sm.tsa.AutoReg(dY[countries[i]].dropna().reset_index(drop=True), lags=1)  
    result = model.fit()  
    res[countries[i]] = result

    MDY[countries[i]] = np.mean(dY[countries[i]])

    RHO[countries[i]] = res[countries[i]].params.iloc[1]
    SE[countries[i]] = res[countries[i]].scale

In [38]:
per = 4
N   = 1000000
e = np.random.randn(per * N)
x = {}

for i in range(n):
    x[countries[i]] = np.std(res[countries[i]].resid) * e


In [39]:
x = {}
SY = {}

for i in range(n):
    x[countries[i]] = np.std(res[countries[i]].resid) * e
    x[countries[i]] = np.sum(np.reshape(x[countries[i]], (per, N)), axis=0)
    x[countries[i]] = x[countries[i]].T
    SY[countries[i]] = np.std(x[countries[i]])

In [40]:
mt = {}
BSY = {}

for i in range(n):
    BSY[countries[i]] = datBY[countries[i]]
    BSY[countries[i]].index = datT["Time"][5:-1]
    #BSY[countries[i]] = daty[countries[i]]

In [41]:
DY = {}
yy = {}
zz = {}

for i in range(n):
    #DY[countries[i]] = np.log(daty[countries[i]][1:]) - np.log(daty[countries[i]][:-1])
    DY[countries[i]] = np.log(daty[countries[i]][1:].values) - np.log(daty[countries[i]][:-1].values)
    #DY[countries[i]] = DY[countries[i]][:-1, :]

    DY[countries[i]] = DY[countries[i]].reshape(-1, 1)  # Omform til todimensjonal matrise
    DY[countries[i]] = DY[countries[i]][:-1, :]  # Fjern den siste raden

    DY[countries[i]] = [item[0] for item in DY[countries[i]]]

In [42]:
with open('variabler.pkl', 'wb') as f:
    pickle.dump([countries, ALP1, ALP2, ALP3, MDY, SY, RHO, SE, BSY, DY], f)